<a href="https://colab.research.google.com/github/saritmaitra/AL-VAR-VECM-/blob/main/exception_handling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyforest
from pyforest import *
import datetime, pickle, copy, warnings
from pandas import DataFrame, merge, concat
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

from google.colab import files
uploaded = files.upload()

  Created wheel for pyforest: filename=pyforest-1.0.3-py2.py3-none-any.whl size=13716 sha256=212a34bb7b8782936330d0502b0bf8de042b6c27a44bb36330b625e37874704a
  Stored in directory: /root/.cache/pip/wheels/af/0b/39/340a7f15fc8d4ff5ab50847b28789afea04452a9d51b9721b5
Successfully built pyforest


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saving AL_final_data.csv to AL_final_data.csv


In [9]:
df = pd.read_csv("AL_final_data.csv")
df.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
df = df.set_index('timestamp')
df.index = pd.to_datetime(df.index)

# spliting train/test
nobs = 10 # keeping 10 mins observations aside for validation
train, test = df[0:- nobs], df[- nobs:]

train_log = np.log(train)
train_trans = train_log.diff().dropna()
print('Description of diff data:'); print(train_trans.describe());

import statsmodels.tsa.api as smt
from statsmodels.tsa.api import VAR

#train_trans = train_trans.reset_index()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Description of diff data:
                 es        eurusd        gbpusd            gc            nq        usdjpy
count  29334.000000  2.933400e+04  2.933400e+04  2.933400e+04  29334.000000  2.933400e+04
mean       0.000002  5.045788e-07 -6.211018e-07  7.151169e-07      0.000003  1.505771e-07
std        0.000387  2.640747e-04  2.716109e-04  3.004687e-04      0.000408  4.423934e-04
min       -0.013492 -1.688387e-02 -1.482272e-02 -5.309362e-03     -0.015196 -4.359512e-02
25%       -0.000171 -4.628531e-05 -8.107760e-05 -1.172024e-04     -0.000168 -5.114816e-05
50%        0.000000  0.000000e+00  0.000000e+00  0.000000e+00      0.000000  0.000000e+00
75%        0.000172  4.634403e-05  8.076134e-05  1.177804e-04      0.000178  5.112694e-05
max        0.008889  1.659153e-02  1.446320e-02  6.787581e-03      0.009019  4.335270e-02


In [3]:
#train_trans = train_trans[['es', 'eurusd', 'gbpusd', 'gc', 'nq', 'usdjpy']]
#train_trans

In [4]:
# pass "1min" frequency
#train_trans.index = pd.DatetimeIndex(train_trans.index).to_period('1min')

# fitting VAR model with associated freq & lags
model = VAR(endog = train_trans.values).fit(maxlags = 8)
# model.summary()

#pd.options.display.float_format = "{:.2f}".format
# Get the lag order
lag_order = model.k_ar
print(lag_order) 

# Input data for forecasting
input_data = train_trans.values[-lag_order:]
print(input_data)

fc = model.forecast(y = input_data, steps = nobs)
#predict = DataFrame(fc, columns = train.columns + '_trans')
#print(predict)

8
[[-2.47739379e-04  1.35066993e-04  6.47387992e-05  1.73190163e-04
  -5.08610526e-04 -1.29818347e-04]
 [-4.12958642e-05 -2.25098987e-05  4.04596213e-05  5.77233895e-05
  -6.52251900e-05 -2.78204100e-05]
 [ 2.06462270e-04  7.65315884e-05  1.17323646e-04 -1.73180166e-04
   5.47759417e-04 -4.63681025e-06]
 [-4.12890440e-05  7.65257318e-05 -2.42727284e-05  5.77300543e-05
   3.91141939e-05 -5.56434000e-05]
 [-2.06470795e-04 -8.55291441e-05 -2.38713216e-04 -2.88646105e-05
  -3.65125320e-04 -1.85484881e-05]
 [ 0.00000000e+00  9.00304754e-05  1.61857476e-05 -5.77317207e-05
  -7.82584878e-05 -1.25211294e-04]
 [ 1.23887593e-04 -2.29593733e-04 -1.13305735e-04 -8.66038308e-05
   1.56510852e-04 -6.95686051e-05]
 [ 0.00000000e+00  2.29593733e-04 -8.09405290e-05 -8.66113317e-05
   2.73835062e-04 -2.55126301e-04]]


In [5]:
df_pred = DataFrame(fc, columns = train_trans.columns + '_1d')
print(df_pred)

          es_1d     eurusd_1d     gbpusd_1d         gc_1d         nq_1d     usdjpy_1d
0 -1.646280e-06 -8.113920e-05  6.653467e-05  1.635449e-05 -1.289316e-05  2.966394e-05
1  1.885735e-05 -9.256758e-06 -4.914509e-06  4.620398e-06  1.947670e-05  1.453923e-06
2 -2.130770e-05 -7.664500e-06 -1.333770e-05  7.872353e-06 -2.344931e-05 -3.167380e-06
3  7.631417e-06 -2.117638e-06  1.501031e-05 -1.015070e-05  1.453891e-05  5.078959e-06
4 -9.063476e-06  7.432817e-06  1.176830e-05  6.442872e-06 -1.747693e-06  3.753441e-06
5  2.358650e-07 -2.208459e-06 -2.039585e-06  5.041519e-06  5.635129e-07 -3.746885e-07
6 -8.249336e-06  4.292135e-07 -4.889857e-06  6.726524e-06 -2.081102e-06 -5.706839e-07
7  1.333822e-06 -2.298150e-06 -6.124405e-06 -4.141979e-07 -1.056524e-06  7.148391e-07
8  4.274447e-06  5.701885e-06  3.459421e-07  7.223630e-07  5.373410e-06  1.092963e-07
9  2.417483e-06 -9.720897e-07  3.215604e-07  6.417956e-07  3.386466e-06 -1.829519e-07


In [11]:
# inverse differenced data
def inverse_diff(train_trans, df_pred):
    """Revert back the differencing to get the forecast to original scale."""
    df_fc = df_pred.copy()
    columns = train_trans.columns
    for col in columns:        
        # Roll back 1st Diff
        df_fc[str(col)+'_fcast'] = train_log[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
    return df_fc

df_results = inverse_diff(train, df_pred)        
z = df_results.loc[:, ['eurusd_fcast', 'gbpusd_fcast', 'es_fcast',
                       'gc_fcast','nq_fcast', 'usdjpy_fcast']]
print('Predicted 10 values:')
print(z)

Predicted 10 values:
   eurusd_fcast  gbpusd_fcast  es_fcast  gc_fcast  nq_fcast  usdjpy_fcast
0     13.920496     14.026992  8.015367  7.456933  9.168190     18.495595
1     13.920487     14.026987  8.015386  7.456938  9.168209     18.495597
2     13.920479     14.026974  8.015365  7.456945  9.168186     18.495594
3     13.920477     14.026989  8.015372  7.456935  9.168200     18.495599
4     13.920485     14.027001  8.015363  7.456942  9.168198     18.495602
5     13.920483     14.026999  8.015363  7.456947  9.168199     18.495602
6     13.920483     14.026994  8.015355  7.456954  9.168197     18.495601
7     13.920481     14.026988  8.015356  7.456953  9.168196     18.495602
8     13.920486     14.026988  8.015361  7.456954  9.168201     18.495602
9     13.920485     14.026988  8.015363  7.456954  9.168205     18.495602


In [13]:
# inverse log transformation
pd.options.display.float_format = "{:.2f}".format
np.exp(z)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,eurusd_fcast,gbpusd_fcast,es_fcast,gc_fcast,nq_fcast,usdjpy_fcast
0,1110694.88,1235507.20,3027.12,1731.83,9587.25,107779197.11
1,1110684.59,1235501.13,3027.18,1731.84,9587.44,107779353.81
2,1110676.08,1235484.65,3027.11,1731.85,9587.21,107779012.43
3,1110673.73,1235503.20,3027.14,1731.83,9587.35,107779559.84
4,1110681.98,1235517.74,3027.11,1731.84,9587.34,107779964.38
5,1110679.53,1235515.22,3027.11,1731.85,9587.34,107779924.00
6,1110680.01,1235509.17,3027.08,1731.86,9587.32,107779862.49
7,1110677.46,1235501.61,3027.09,1731.86,9587.31,107779939.54
8,1110683.79,1235502.04,3027.10,1731.86,9587.36,107779951.32
9,1110682.71,1235502.43,3027.11,1731.87,9587.40,107779931.60
